## Environment Set Up

In [1]:
import os
from pycelonis import get_celonis
import pandas as pd
import json
import re
import shutil
import subprocess
import zipfile
import yaml
import numpy as np
from datetime import datetime
from pycelonis.ems import ColumnTransport, ColumnType
import traceback

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
url = os.environ['CELONIS_URL'] = 'https://lineage.develop.celonis.cloud'
#API Key for this specific url
api_token = os.environ['CELONIS_API_TOKEN'] = 'MDE5YTE2M2UtZmU5YS03NTFkLWFjYmYtZGQ0NWQxODJjZmYzOmc2ZTY2UUkyU3R2RFkxVTA2L3VNc0tiZUxaVmZneHR0RVRuVS9ETFJWS3or'

In [4]:
#Initializing Celonis object
c = get_celonis(base_url = url, api_token = api_token) # adjust base_url and api_token accordingly

KeyType is not set. Defaulted to 'USER_KEY'.


## Studio Lineage (Views + Knowledge Model)

In [7]:
def get_knowledge_model_key(view_node):
    km_key = json.loads(view_node.serialized_content)['metadata'].get('knowledgeModelKey')
    return km_key

def get_data_model_variable(knowledge_model):
    try:
        serialized = json.loads(knowledge_model.serialized_content)
        data_model_expr = serialized.get("dataModelId")
        match = None
        if isinstance(data_model_expr, str):
            match = re.match(r'\${{([a-zA-Z0-9_]+)}}', data_model_expr)
        if not match:
            print('ERROR: No valid Data Model Variable')
            return None
        return match.group(1)
    except (KeyError, json.JSONDecodeError, AttributeError) as e:
        print('ERROR: No Data Model Assigned')
        return None

data_pools = c.data_integration.get_data_pools()

def find_data_pool_id(data_model_id):
    data_pool_id = None
    for data_pool in data_pools:
        data_models = data_pool.get_data_models()
        for data_model in data_models:
            if data_model.id == data_model_id:
                data_pool_id = data_pool.id
                break
    return data_pool_id

In [8]:
# Lineage API call
def get_lineage(celonis, knowledge_model_key):
    try:
        lineage = celonis.client.request(
            method='get',
            url=f'/semantic-layer/api/usage/by-semantic-model/{knowledge_model_key}',
            parse_json =True
        )
        return lineage
    except Exception as e:
        print(e)
        return {}

In [8]:
""" def get_km_attribute_details(knowledge_model):

    Extracts attribute and KPI details from KM serialized_content.
    Returns two dicts: attribute_details and kpi_details

    attribute_details = {}
    kpi_details = {}
    
    try:
        km_content = json.loads(knowledge_model.serialized_content)
        
        # Process record attributes
        if 'records' in km_content:
            for record in km_content['records']:
                record_id = record.get('id')
                
                if 'attributes' not in record:
                    continue
                
                for attr in record['attributes']:
                    attr_id = attr.get('id')
                    pql = attr.get('pql')
                    
                    # Create unique key
                    key = f"{record_id}.{attr_id}".lower()
                    
                    # Determine source type
                    if pql and pql.strip():
                        source_type = 'CUSTOMIZED'
                    else:
                        source_type = 'AUTO_GENERATED'
                    
                    attribute_details[key] = {
                        'source_type': source_type,
                        'pql': pql if pql else ''
                    }
        
        # Process KPIs 

        if 'kpis' in km_content:
            
            for i, kpi in enumerate(km_content['kpis']):
                kpi_id = kpi.get('id')
                kpi_pql = kpi.get('pql', '')
                kpi_name = kpi.get('displayName', kpi_id)
                
                if kpi_id:
                    kpi_details[kpi_id] = {
                        'name': kpi_name,
                        'pql': kpi_pql,
                        'format': kpi.get('format', '')
                    }
        else:
            print("No KPIs found in knowledge model content")
            
    
    except Exception as e:
        print(f"Warning: Could not extract details: {e}")
        import traceback
        traceback.print_exc()
    
    return attribute_details, kpi_details """

' def get_km_attribute_details(knowledge_model):\n\n    Extracts attribute and KPI details from KM serialized_content.\n    Returns two dicts: attribute_details and kpi_details\n\n    attribute_details = {}\n    kpi_details = {}\n\n    try:\n        km_content = json.loads(knowledge_model.serialized_content)\n\n        # Process record attributes\n        if \'records\' in km_content:\n            for record in km_content[\'records\']:\n                record_id = record.get(\'id\')\n\n                if \'attributes\' not in record:\n                    continue\n\n                for attr in record[\'attributes\']:\n                    attr_id = attr.get(\'id\')\n                    pql = attr.get(\'pql\')\n\n                    # Create unique key\n                    key = f"{record_id}.{attr_id}".lower()\n\n                    # Determine source type\n                    if pql and pql.strip():\n                        source_type = \'CUSTOMIZED\'\n                    else:\n     

In [9]:
def get_km_attribute_details(knowledge_model, data_model): 
    """
    Extracts attribute details and finds the 'table_schema' for each table.
    FIXED: Manually constructs schema string to avoid 'bound method' errors.
    """
    attribute_details = {}
    kpi_details = {}
    
    try:
        km_content = json.loads(knowledge_model.serialized_content)
        
        # --- Create a lookup map for table properties ---
        table_properties = {}
        if data_model:
            pool_id = find_data_pool_id(data_model.id)
            for table in data_model.get_tables():
                # SAFELY construct the schema string
                # This matches your Backend logic: pool_id + "_" + data_source_id
                if table.data_source_id:
                    safe_schema = f"{pool_id}_{table.data_source_id}"
                else:
                    safe_schema = pool_id

                # Store schema by lowercase table alias
                table_properties[table.alias_or_name.lower()] = {
                    'table_schema': safe_schema
                }
        
        # Process record attributes
        if 'records' in km_content:
            for record in km_content['records']:
                record_id = record.get('id') # This is the table alias
                table_info = table_properties.get(record_id.lower(), {})
                
                if 'attributes' not in record: continue
                
                for attr in record['attributes']:
                    attr_id = attr.get('id')
                    pql = attr.get('pql')
                    key = f"{record_id}.{attr_id}".lower()
                    
                    if pql and pql.strip(): source_type = 'CUSTOMIZED'
                    else: source_type = 'AUTO_GENERATED'
                    
                    attribute_details[key] = {
                        'source_type': source_type,
                        'pql': pql if pql else '',
                        'table_schema': table_info.get('table_schema', '') # Defaults to empty string, not None
                    }
        
        # Process KPIs 
        if 'kpis' in km_content:
            for i, kpi in enumerate(km_content['kpis']):
                kpi_id = kpi.get('id')
                kpi_pql = kpi.get('pql', '')
                kpi_name = kpi.get('displayName', kpi_id)
                if kpi_id:
                    kpi_details[kpi_id] = {'name': kpi_name, 'pql': kpi_pql, 'format': kpi.get('format', '')}
            
    except Exception as e:
        print(f"Warning: Could not extract details: {e}")
        traceback.print_exc()
    
        
    return attribute_details, kpi_details

In [10]:
def update_with_view_usages(lineage, usage: list, metadata: dict, attribute_details: dict, kpi_details: dict):
    ## View usages
    
    # Record --> View
    parent_key = 'viewUsages'
    key = "recordAttributeReferences"
    for record, attributes in lineage[parent_key][key].items():
        for attribute, views in attributes.items():
            # Get attribute details
            attr_key = f"{record}.{attribute}".lower()
            attr_info = attribute_details.get(attr_key, {'source_type': 'UNKNOWN', 'pql': ''})
            
            for view in views:
                usage.append({
                    'UNIQUE_SOURCE_ID': f'{metadata.get('knowledge_model_id')}.{record}.{attribute}'.lower(),
                    'UNIQUE_TARGET_ID': f'{metadata.get("root_id")}.{view.get("nodeId")}'.lower(),
                    'SOURCE_ID': f'{record}.{attribute}', 
                    'SOURCE_NAME': record,
                    'SOURCE_ATTRIBUTE': attribute,
                    'SOURCE_NODE_TYPE': 'ATTRIBUTES',
                    'SOURCE_TYPE': attr_info['source_type'],
                    'SOURCE_PQL': attr_info['pql'],
                    'SOURCE_STUDIO_ASSET_ID': metadata.get("knowledge_model_id"),
                    'SOURCE_STUDIO_ASSET_TYPE': 'KNOWLEDGE_MODEL',
                    'TARGET_ID': view.get('nodeId'),
                    'TARGET_NAME': view.get('assetName'),
                    'TARGET_ATTRIBUTE': None,
                    'TARGET_NODE_TYPE': 'VIEW',
                    'TARGET_STUDIO_ASSET_ID': view.get('nodeId'),
                    'TARGET_STUDIO_ASSET_TYPE': 'VIEW',
                    'DATA_SOURCE_ID': '',
                    'KNOWLEDGE_MODEL_KEY': metadata.get('knowledge_model_key'),
                    'KNOWLEDGE_MODEL_ID': metadata.get("knowledge_model_id"),
                    'DATA_MODEL_ID': metadata.get('data_model_id'),
                    'DATA_MODEL_NAME': metadata.get('data_model_name'),
                    'DATA_POOL_ID': metadata.get('data_pool_id'),
                    'DATA_POOL_NAME': metadata.get('data_pool_name')
                })

    # KPI --> View
    key = 'kpiReferences'
    
    for kpi, views in lineage[parent_key][key].items():
        kpi_info = kpi_details.get(kpi, {'name': kpi, 'pql': ''})
        
        for view in views:
            usage.append({
                'UNIQUE_SOURCE_ID': f'{metadata.get('knowledge_model_id')}.{kpi}'.lower(),
                'UNIQUE_TARGET_ID': f'{metadata.get("root_id")}.{view.get("nodeId")}'.lower(),
                'SOURCE_ID': f'{kpi}', 
                'SOURCE_NAME': kpi,
                'SOURCE_ATTRIBUTE': None,
                'SOURCE_NODE_TYPE': 'KPIS',
                'SOURCE_TYPE': 'KPI',
                'SOURCE_PQL': kpi_info['pql'],
                'SOURCE_STUDIO_ASSET_ID': metadata.get("knowledge_model_id"),
                'SOURCE_STUDIO_ASSET_TYPE': 'KNOWLEDGE_MODEL',
                'TARGET_ID': view.get('nodeId'),
                'TARGET_NAME': view.get('assetName'),
                'TARGET_NODE_TYPE': 'VIEW',
                'TARGET_STUDIO_ASSET_ID': view.get('nodeId'),
                'TARGET_STUDIO_ASSET_TYPE': 'VIEW',
                'DATA_SOURCE_ID': '',
                'KNOWLEDGE_MODEL_KEY': metadata.get('knowledge_model_key'),
                'KNOWLEDGE_MODEL_ID': metadata.get("knowledge_model_id"),
                'DATA_MODEL_ID': metadata.get('data_model_id'),
                'DATA_MODEL_NAME': metadata.get('data_model_name'),
                'DATA_POOL_ID': metadata.get('data_pool_id'),
                'DATA_POOL_NAME': metadata.get('data_pool_name')
            })
    
    return usage

def update_with_km_usages(lineage, usage, metadata, attribute_details: dict, kpi_details: dict):
    ## Knowledge Model usages
    
    # Record --> KM property
    parent_key = 'knowledgeModelUsages'
    key = "recordAttributeUsages"
    properties = ['kpis', 'filters', 'attributes', 'flags']
    map_id = {'kpis': 'id', 'filters': 'id', 'attributes': 'recordId', 'flags': 'id'}

    for property in properties:
        for record, attributes in lineage[parent_key][key][property].items():
            for attribute, props in attributes.items():
                # Get attribute details
                attr_key = f"{record}.{attribute}".lower()
                attr_info = attribute_details.get(attr_key, {'source_type': 'UNKNOWN', 'pql': ''})
                
                for prop in props:
                    prop_id = prop.get(map_id.get(property), 'id')
                    prop_id = f'{prop_id}.{prop.get("attributeId")}' if prop.get('attributeId') else prop_id
                    usage.append({
                        'UNIQUE_SOURCE_ID': f'{metadata.get('knowledge_model_id')}.{record}.{attribute}'.lower(),
                        'UNIQUE_TARGET_ID': f'{metadata.get("knowledge_model_id")}.{prop_id}'.lower(),
                        'SOURCE_ID': f'{record}.{attribute}'.lower(), 
                        'SOURCE_NAME': record,
                        'SOURCE_ATTRIBUTE': attribute,
                        'SOURCE_NODE_TYPE': 'ATTRIBUTES',
                        'SOURCE_TYPE': attr_info['source_type'],
                        'SOURCE_PQL': attr_info['pql'],
                        'SOURCE_STUDIO_ASSET_ID': metadata.get("knowledge_model_id"),
                        'SOURCE_STUDIO_ASSET_TYPE': 'KNOWLEDGE_MODEL',
                        'TARGET_ID': f'{prop_id}'.lower(),
                        'TARGET_NAME': prop.get('displayName'),
                        'TARGET_ATTRIBUTE': prop.get('attributeId'),
                        'TARGET_NODE_TYPE': property.upper(),
                        'TARGET_STUDIO_ASSET_ID': metadata.get("knowledge_model_id"),
                        'TARGET_STUDIO_ASSET_TYPE':'KNOWLEDGE_MODEL',
                        'DATA_SOURCE_ID': '',
                        'KNOWLEDGE_MODEL_KEY': metadata.get('knowledge_model_key'),
                        'KNOWLEDGE_MODEL_ID': metadata.get("knowledge_model_id"),
                        'DATA_MODEL_ID': metadata.get('data_model_id'),
                        'DATA_MODEL_NAME': metadata.get('data_model_name'),
                        'DATA_POOL_ID': metadata.get('data_pool_id'),
                        'DATA_POOL_NAME': metadata.get('data_pool_name')
                    })
    
    # KPI --> KM property
    key = 'kpiUsages'
    for property in properties:
        for kpi, props in lineage[parent_key][key][property].items():
            kpi_info = kpi_details.get(kpi, {'name': kpi, 'pql': ''})
            
            for prop in props:
                prop_id = prop.get(map_id.get(property), 'id')
                prop_id = f'{prop_id}.{prop.get("attributeId")}' if prop.get('attributeId') else prop_id
                usage.append({
                        'UNIQUE_SOURCE_ID': f'{metadata.get('knowledge_model_id')}.{kpi}'.lower(),
                        'UNIQUE_TARGET_ID': f'{metadata.get("knowledge_model_id")}.{prop_id}'.lower(),
                        'SOURCE_ID': f'{kpi}'.lower(), 
                        'SOURCE_NAME': kpi,
                        'SOURCE_ATTRIBUTE': None,
                        'SOURCE_NODE_TYPE': 'KPIS',
                        'SOURCE_TYPE': 'KPI',
                        'SOURCE_PQL': kpi_info['pql'],
                        'SOURCE_STUDIO_ASSET_ID': metadata.get("knowledge_model_id"),
                        'SOURCE_STUDIO_ASSET_TYPE': 'KNOWLEDGE_MODEL',
                        'TARGET_ID': f'{prop_id}'.lower(),
                        'TARGET_NAME': prop.get('displayName'),
                        'TARGET_ATTRIBUTE': prop.get('attributeId'),
                        'TARGET_NODE_TYPE': property.upper(),
                        'TARGET_STUDIO_ASSET_ID': metadata.get("knowledge_model_id"),
                        'TARGET_STUDIO_ASSET_TYPE':'KNOWLEDGE_MODEL',
                        'DATA_SOURCE_ID': '',
                        'KNOWLEDGE_MODEL_KEY': metadata.get('knowledge_model_key'),
                        'KNOWLEDGE_MODEL_ID': metadata.get("knowledge_model_id"),
                        'DATA_MODEL_ID': metadata.get('data_model_id'),
                        'DATA_MODEL_NAME': metadata.get('data_model_name'),
                        'DATA_POOL_ID': metadata.get('data_pool_id'),
                        'DATA_POOL_NAME': metadata.get('data_pool_name')
                    })
                
    return usage

## Bridge Table - Integration between backend and frontend

In [11]:
def create_bridge_links(data_model, knowledge_model_id, pool_table_lookup):
    """
    Creates bridge links.
    1. Raw Node: Lowercase (to match Backend physical tables)
    2. Prefixed Node: Case-sensitive (to preserve Alias for Frontend)
    """
    links = []
    try:
        dm_id = data_model.id
        data_pool_id = find_data_pool_id(dm_id)
        data_pool = c.data_integration.get_data_pool(data_pool_id)
        
        for table in data_model.get_tables():
            # 1. Get the Logical Name (Alias)
            dm_table_alias = table.alias_or_name
            
            # 2. Look up the PHYSICAL info
            # We search using lowercase because your dictionary keys are lowercase
            physical_info = pool_table_lookup.get(table.name.lower())
            
            if physical_info:
                # Get the schema and name from the dictionary
                dict_schema = physical_info['schema'].lower()
                dict_name = physical_info['name'].lower()
                
                # --- BUILD SOURCE NODES ---
                
                # Variant 1: Raw Physical -> FORCE LOWERCASE
                # This ensures 'BUSINESS_GRAPH_SCHEMA' becomes 'business_graph_schema'
                # to match your Backend CSV.
                source_node_raw = f"{dict_schema}_{dict_name}"
                
                # Variant 2: Prefixed with ID -> PRESERVE CASE
                # We do NOT use .lower() here.
                # It uses the schema from the dict (likely Upper) and the Alias (Mixed)
                source_node_prefixed = f"DATA_MODEL_TABLE_{dict_schema}_{dm_id}_{dm_table_alias}"
                
            else:
                # Fallback if physical table not found
                continue

            # 3. Create Links for Every Column
            for column in table.get_columns():
                
                # Build Target (Frontend) Node ID
                # This matches your frontend script logic (typically lowercased)
                target_node = f"{knowledge_model_id}.{dm_table_alias}.{column.name}".lower()
                
                link_data = {
                    "target_node": target_node,
                    "task_target": "DATA_MODEL_COLUMN",
                    "data_pool_id": data_pool_id,
                    "data_pool_name": data_pool.name,
                    "data_schema_id": dm_id, 
                    "data_schema_name": data_model.name
                }
                
                # Add BOTH links
                links.append({"source_node": source_node_raw, **link_data})
                links.append({"source_node": source_node_prefixed, **link_data})
                
    except Exception as e:
        print(f"    Warning: Could not create bridge links for DM {data_model.name}: {e}")
    
    return links

## Execution Block - Scan the environment and generate lineage and link table

In [14]:
# Execution block - scanning all environment KMs
usage = []
all_bridge_links = []
all_metadata = []
error_km_count = 0
processed_km_count = 0

for space in c.studio.get_spaces():
    print("Space", space.name)
    for package in space.get_packages():
        print("  Package", package.name)

        # Safely get Knowledge Models
        try:

            knowledge_models = package.get_knowledge_models()
        except Exception as e:
            print(f"    !!! CRITICAL ERROR: Could not fetch KMs for package '{package.name}'. Skipping package.")
            print(f"    Error details: {e}")
            error_km_count += 1
            continue # Skip to the next package
        
        for knowledge_model in package.get_knowledge_models():
            try:
                print("\tKnowledge Model", knowledge_model.name)
                knowledge_model_key = knowledge_model.key
                knowledge_model_id = knowledge_model.id
                
                # 1. Find Data Model ID
                dm_variable = get_data_model_variable(knowledge_model)
                if not dm_variable:
                    print(f"\t\t No data model variable found, skipping")
                    error_km_count += 1
                    continue
                
                try:
                    package_variables = package.get_variables()
                    variable_obj = package_variables.find(dm_variable, "key")
                    data_model_id = variable_obj.value
                except Exception as e:
                    print(f"\t\t Error finding variable '{dm_variable}': {str(e)}")
                    error_km_count += 1
                    continue
                    
                # 2. Find Data Pool & Data Model
                data_pool_id = find_data_pool_id(data_model_id)
                if data_pool_id:
                    data_pool = c.data_integration.get_data_pool(data_pool_id)
                    data_pool_name = data_pool.name
                    data_model = data_pool.get_data_model(data_model_id)
                    data_model_name = data_model.name
                    
                    # --- 3. Build Physical Table Lookup for this Pool (NEW) ---
                    pool_table_lookup = {}
                    try:
                        for pool_table in data_pool.get_tables():
                            # Robust Schema Logic
                            if hasattr(pool_table, 'schema_name') and pool_table.schema_name:
                                phys_schema = pool_table.schema_name
                            else:
                                phys_schema = f"{data_pool.id}_{pool_table.data_source_id}"
                            
                            # Store Lowercase Name -> Schema/Name info
                            pool_table_lookup[pool_table.name.lower()] = {
                                'schema': phys_schema,
                                'name': pool_table.name
                            }
                    except Exception:
                        pass 
                    # ----------------------------------------------------

                    # --- 4. CREATE BRIDGE LINKS (With Lookup) ---
                    # This creates the backend-matching links
                    bridge_links = create_bridge_links(data_model, knowledge_model_id, pool_table_lookup)
                    all_bridge_links.extend(bridge_links)

                else:
                    print(f"\t\t Could not find data pool for data model ID: {data_model_id}")
                    error_km_count += 1
                    continue

                # 5. Create Metadata Dictionary
                metadata = {
                    'data_model_id': data_model_id, 
                    'data_model_name': data_model_name,
                    'data_pool_id': data_pool_id,
                    'data_pool_name': data_pool_name,
                    'root_id': package.id, 
                    'space_id': space.id,
                    'knowledge_model_key': knowledge_model_key,
                    'knowledge_model_id': knowledge_model_id
                }

                # Extract attribute and KPI details from KM
                attribute_details, kpi_details = get_km_attribute_details(knowledge_model, data_model)
                
                # Get lineage
                lineage = get_lineage(c, knowledge_model.root_with_key)
                
                # Check if lineage has required keys
                if not lineage or 'viewUsages' not in lineage or 'knowledgeModelUsages' not in lineage:
                    print(f"\t\t Missing lineage data, skipping")
                    error_km_count += 1
                    continue
                
                
                update_with_view_usages(lineage, usage, metadata, attribute_details, kpi_details)
                update_with_km_usages(lineage, usage, metadata, attribute_details, kpi_details)

                all_metadata.append(metadata)
                processed_km_count += 1
                print(f"\t\t Successfully processed")
                
            except Exception as e:
                print(f"\t\tError processing knowledge model: {str(e)}")
                error_km_count += 1

print(f"\nProcessing complete!")
print(f"Successfully processed {processed_km_count} knowledge models")
print(f"Skipped {error_km_count} knowledge models due to errors")
            
df_studio = pd.DataFrame(usage)
df_studio.to_csv('lineage_studio.csv')
print("Saved lineage_studio.csv")

# Create the bridge lineage CSV
df_bridge = pd.DataFrame(all_bridge_links).drop_duplicates()
df_bridge.to_csv('bridge_lineage_mapping.csv', index=False)
print("Saved bridge_lineage_mapping.csv")

Space ahilmer - lineage test
  Package Monitoring
	Knowledge Model Replication Cockpit Monitoring KM
ERROR: No valid Data Model Variable
		 No data model variable found, skipping
	Knowledge Model Data Pipeline Monitoring KM
ERROR: No valid Data Model Variable
		 No data model variable found, skipping
	Knowledge Model Data Consumption Monitoring KM
ERROR: No valid Data Model Variable
		 No data model variable found, skipping
Space Flo
  Package Monitoring
	Knowledge Model Data Consumption Monitoring KM
ERROR: No valid Data Model Variable
		 No data model variable found, skipping
	Knowledge Model Data Pipeline Monitoring KM
ERROR: No valid Data Model Variable
		 No data model variable found, skipping
	Knowledge Model Replication Cockpit Monitoring KM
ERROR: No valid Data Model Variable
		 No data model variable found, skipping
Space FS
  Package Data Lineage
	Knowledge Model KM - Data Lineage
		 Successfully processed
Space Paula's Space
  Package Data Lineage - Test
	Knowledge Model KM


## Data Models Loads - Extract all data

In [ ]:
all_metadata

[{'data_model_id': 'da1f57a6-3485-4eb3-944d-12825f56687e',
  'data_model_name': 'Data Lineage Monitoring',
  'data_pool_id': '831db6ab-da5e-4d57-9967-c97f4320d2d6',
  'data_pool_name': 'Monitoring Pool',
  'root_id': '34ca0a04-a73c-4ea8-ba73-f1863cf8471d',
  'space_id': '24b90297-f60c-4ad2-b700-87ad401b9961',
  'knowledge_model_key': 'km-data-lineage',
  'knowledge_model_id': 'dd35ad1d-5ba1-44dc-b9a0-533587643b35'},
 {'data_model_id': '1e846159-f3a9-42a3-9d91-b3c7c0c7f904',
  'data_model_name': 'data lineage - test',
  'data_pool_id': 'ebdcfc35-b1c0-4a8c-bf5c-ae3c5f995bab',
  'data_pool_name': 'test pool - data lineage app [PC]',
  'root_id': 'c9b98018-21f1-4f3b-86b0-e1ba393125a5',
  'space_id': '4e7bf097-7bea-48e5-af3d-035576e47309',
  'knowledge_model_key': 'dm_data_lineage_test-km',
  'knowledge_model_id': '59b95b69-c4b2-4696-b4a7-b1a4cf1235b5'},
 {'data_model_id': 'da1f57a6-3485-4eb3-944d-12825f56687e',
  'data_model_name': 'Data Lineage Monitoring',
  'data_pool_id': '831db6ab-da5

In [15]:
# Create a unique set of data model IDs we need to find
unique_data_model_ids = {item['data_model_id'] for item in all_metadata}
print(f"Found {len(unique_data_model_ids)} unique Data Models to scan for.")
unique_data_model_ids

Found 8 unique Data Models to scan for.


{'088283c6-9918-4b59-9a4b-96d9cb38bb15',
 '0ad397d5-ca2a-4a57-a0ab-76dc042117a0',
 '1e846159-f3a9-42a3-9d91-b3c7c0c7f904',
 '24562740-9c61-4566-93ac-21f23f95a157',
 '4ef2ec4b-aa72-4896-b53c-3d678c9683b5',
 '80fea3cc-e1ba-4bbf-a29a-da1d2e50557f',
 '904789ed-fa95-4ee4-81fa-b1cb1a8987e2',
 'da1f57a6-3485-4eb3-944d-12825f56687e'}

In [16]:
# Create a unique set of data model IDs we need to find
unique_KM_ids = {item['knowledge_model_id'] for item in all_metadata}
print(f"Found {len(unique_KM_ids)} unique Knowledge Models to scan for.")
unique_KM_ids

Found 11 unique Knowledge Models to scan for.


{'423975bc-05cb-45fe-b0e3-58cb7cd1c4ed',
 '4d63f4e9-1e92-454e-a128-f161f43f18d6',
 '59b95b69-c4b2-4696-b4a7-b1a4cf1235b5',
 '6fb03d9f-d533-4eb9-bcd3-3560729240f5',
 '75b1bc36-70b5-4f57-91d7-bbdef6ed410c',
 '75fa6a46-61c2-455b-b04b-c4e42762ce5a',
 '7de213a8-ba42-4b92-8d16-ab35f98a8b38',
 'b2afef54-36f0-438c-9074-f92929c0b606',
 'bc107876-ed37-4c2c-be70-a2bfea3c36c3',
 'dba5e290-fa7e-463e-8d1f-267762c01a41',
 'dd35ad1d-5ba1-44dc-b9a0-533587643b35'}

In [30]:
def get_datamodel_metadata(celonis_connection):
    """
    Scans for the required Data Models and returns a DataFrame of all their
    tables and columns. Iterates through all metadata but only
    processes each unique Data Model once.
    """
    print("Starting Data Model Scan")
    data_model_data = []
    
    # A set to keep track of the Data Model IDs we have already processed
    processed_data_model_ids = set()

    print("\nExtracting tables and columns from Data Models")
    # Loop through all the metadata
    for metadata in all_metadata:
        try:
            current_dm_id = metadata['data_model_id']
            
            # If we have already processed this Data Model ID, skip to the next item
            if current_dm_id in processed_data_model_ids:
                continue

            # If it's a new ID, process it
            data_pool = celonis_connection.data_integration.get_data_pool(metadata['data_pool_id'])
            data_model = data_pool.get_data_model(current_dm_id)
            
            for table in data_model.get_tables():
                for column in table.get_columns():
                    # Create unique_id from d_model_id.table_name.column_name (lowercase)
                    unique_id = f"{data_model.id}.{table.name}.{column.name}".lower() 

                    data_model_data.append({
                        "unique_id": unique_id,
                        "d_pool_id": data_pool.id,
                        "d_pool_name": data_pool.name,
                        "d_model_id": data_model.id,
                        "d_model_name": data_model.name,
                        "table_name": table.name,
                        "column_name": column.name
                })
            
            # After successfully processing, add the ID to our set of processed IDs
            processed_data_model_ids.add(current_dm_id)

        except Exception as e:
            # Using a more specific variable for the error message
            dm_id_for_error = metadata.get('data_model_id', 'unknown')
            print(f"Warning: Could not process tables for Data Model ID {dm_id_for_error}: {e}")
            continue
            
    print(f"\nData Model scan complete. Found {len(data_model_data)} columns in total.")
    
    return pd.DataFrame(data_model_data)

In [31]:
df_data_models = get_datamodel_metadata(c)
df_data_models.to_csv('data_models.csv')


Starting Data Model Scan

Extracting tables and columns from Data Models

Data Model scan complete. Found 1630 columns in total.


## JOIN Data Models data with Studio Lineage Data

In [32]:
## JOIN Data Models data with Studio Lineage Data

# Add JOIN_KEY to df_studio
df_studio['JOIN_KEY'] = (
    df_studio['DATA_MODEL_ID'] + '.' + 
    df_studio['SOURCE_NAME'] + '.' + 
    df_studio['SOURCE_ATTRIBUTE']
).str.lower()

print(f"df_studio rows: {len(df_studio)} rows")

# Use existing unique_id from df_data_models (no need to create JOIN_KEY)
print(f"\ndf_data_models rows: {len(df_data_models)}")
print(f"Unique IDs in df_data_models: {df_data_models['unique_id'].nunique()}")

# OUTER JOIN using unique_id from df_data_models and JOIN_KEY from df_studio
df_final = pd.merge(
    df_data_models,
    df_studio,
    left_on='unique_id',
    right_on='JOIN_KEY',
    how='outer',  # OUTER JOIN keeps all rows from BOTH dataframes
    indicator=True,
    suffixes=('_dm', '_studio')  # Add suffixes to handle duplicate column names
)

# Create USED/NOT_USED column based on merge indicator
df_final['USED_NOT_USED'] = df_final['_merge'].map({
    'both': 'USED',              # Data model column is used in studio
    'left_only': 'NOT_USED',     # Data model column not used
    'right_only': 'USED'         # Studio row without data model match (still used in lineage)
})

# Drop the merge indicator and JOIN_KEY columns (keep unique_id)
df_final = df_final.drop(columns=['_merge', 'JOIN_KEY'])

# Reorder columns: 
# 1. All original data_models columns (including unique_id) - with _dm suffix if duplicated
# 2. USED_NOT_USED column
# 3. ALL studio columns (keeping everything, no exclusions)

# Get all data model columns (unique_id is already included)
data_model_cols = list(df_data_models.columns)
# Update to use _dm suffix for columns that exist in both dataframes
data_model_cols_final = []
for col in data_model_cols:
    if col + '_dm' in df_final.columns:
        data_model_cols_final.append(col + '_dm')
    else:
        data_model_cols_final.append(col)

# Get ALL studio columns (excluding only JOIN_KEY which we dropped)
studio_cols = [col for col in df_studio.columns if col != 'JOIN_KEY']
# Update to use _studio suffix for duplicated columns
studio_cols_final = []
for col in studio_cols:
    if col + '_studio' in df_final.columns:
        studio_cols_final.append(col + '_studio')
    else:
        studio_cols_final.append(col)

# Final column order: data_models columns + USED_NOT_USED + ALL studio columns
final_columns = data_model_cols_final + ['USED_NOT_USED'] + studio_cols_final

# Reorder columns
df_final = df_final[final_columns]

# Verify row counts
print(f"\n=== Verification ===")
print(f"Original df_data_models rows: {len(df_data_models)}")
print(f"Original df_studio rows: {len(df_studio)}")
print(f"Final df_final rows: {len(df_final)}")

# Summary statistics
print(f"\n=== Final Data Model Usage Report ===")
print(f"Total rows: {len(df_final)}")
print(f"\nUsage breakdown:")
print(df_final['USED_NOT_USED'].value_counts())

# Calculate usage percentage
used_count = (df_final['USED_NOT_USED'] == 'USED').sum()
not_used_count = (df_final['USED_NOT_USED'] == 'NOT_USED').sum()
total_count = len(df_final)

print(f"\nUsage Statistics:")
print(f"- Columns/Rows USED: {used_count}")
print(f"- Data Model columns NOT_USED: {not_used_count}")
print(f"- Total rows: {total_count}")

# Show breakdown by data model presence
# Check which column name was used (with or without suffix)
dm_id_col = 'd_model_id_dm' if 'd_model_id_dm' in df_final.columns else 'd_model_id'
data_model_present = df_final[dm_id_col].notna().sum()
data_model_missing = df_final[dm_id_col].isna().sum()

print(f"\nData Model Coverage:")
print(f"- Rows with Data Model info: {data_model_present}")
print(f"- Rows without Data Model info (studio only): {data_model_missing}")

# Save the final result
df_final.to_csv('tables_usage_report.csv', index=False)
print("\n Final complete lineage report saved to 'tables_usage_report.csv'")

df_studio rows: 1873 rows

df_data_models rows: 1630
Unique IDs in df_data_models: 1566

=== Verification ===
Original df_data_models rows: 1630
Original df_studio rows: 1873
Final df_final rows: 3271

=== Final Data Model Usage Report ===
Total rows: 3271

Usage breakdown:
USED_NOT_USED
USED        1873
NOT_USED    1398
Name: count, dtype: int64

Usage Statistics:
- Columns/Rows USED: 1873
- Data Model columns NOT_USED: 1398
- Total rows: 3271

Data Model Coverage:
- Rows with Data Model info: 2499
- Rows without Data Model info (studio only): 772

 Final complete lineage report saved to 'tables_usage_report.csv'


In [33]:
df_studio['NODE_TYPE_COMBINED'] = (
    df_studio['SOURCE_NODE_TYPE'].astype(str) + '-' + 
    df_studio['SOURCE_TYPE'].astype(str)
)

# Create complete source nodes list with all columns from studio
df_source_nodes = df_studio[[
    'UNIQUE_SOURCE_ID', 
    'SOURCE_ID', 
    'NODE_TYPE_COMBINED',
    'SOURCE_STUDIO_ASSET_TYPE',
    'SOURCE_STUDIO_ASSET_ID',
    'DATA_MODEL_ID',
    'DATA_MODEL_NAME',
    'DATA_POOL_ID',
    'DATA_POOL_NAME'
]].rename(columns={
    'UNIQUE_SOURCE_ID': 'node',
    'SOURCE_ID': 'node_name',
    'DATA_POOL_ID': 'data_pool_id',
    'DATA_POOL_NAME': 'data_pool_name',
    'DATA_MODEL_ID': 'data_model_id',
    'DATA_MODEL_NAME': 'data_model_name',
    'NODE_TYPE_COMBINED': 'category',
    'SOURCE_STUDIO_ASSET_TYPE': 'asset_type',
    'SOURCE_STUDIO_ASSET_ID': 'asset_id',
})

# Create complete target nodes list
df_target_nodes = df_studio[[
    'UNIQUE_TARGET_ID', 
    'TARGET_NAME',
    'TARGET_NODE_TYPE', 
    'TARGET_STUDIO_ASSET_TYPE',
    'TARGET_STUDIO_ASSET_ID',
    'DATA_MODEL_ID',
    'DATA_MODEL_NAME',
    'DATA_POOL_ID',
    'DATA_POOL_NAME'
]].rename(columns={
    'UNIQUE_TARGET_ID': 'node',
    'TARGET_NAME': 'node_name',
    'DATA_POOL_ID': 'data_pool_id',
    'DATA_POOL_NAME': 'data_pool_name',
    'DATA_MODEL_ID': 'data_model_id',
    'DATA_MODEL_NAME': 'data_model_name',
    'TARGET_NODE_TYPE': 'category',
    'TARGET_STUDIO_ASSET_TYPE': 'asset_type',
    'TARGET_STUDIO_ASSET_ID': 'asset_id',
})

# Combine ALL nodes from lineage
df_mapping_nodes = pd.concat([df_source_nodes, df_target_nodes]).drop_duplicates(subset=['node']).reset_index(drop=True)

""" # Now do outer join - this will keep ALL lineage nodes AND all data model columns
df_network_nodes = pd.merge(
    df_lineage_nodes,
    df_data_models,
    left_on='NODE_ID',
    right_on='unique_id',
    how='outer'
)

# Fill in missing values
df_network_nodes['NODE_ID'] = df_network_nodes['NODE_ID'].fillna(df_network_nodes['unique_id'])
df_network_nodes['NODE_NAME'] = df_network_nodes['NODE_NAME'].fillna(df_network_nodes['column_name'])
df_network_nodes['NODE_TYPE'] = df_network_nodes['NODE_TYPE'].fillna('DATA_COLUMN')

# Use UNIQUE_KEY instead of JOIN_KEY (which was dropped)
df_network_nodes['IS_USED'] = np.where(
    df_network_nodes['unique_id'].notna(),
    df_network_nodes['unique_id'].isin(df_studio['unique_key']),  # Fix: uppercase 'K',  
    True  # Non-data-model nodes are inherently "used" if they're in lineage
) """

# Save and show stats
df_mapping_nodes.to_csv('mapping_nodes_studio.csv')

In [34]:
df_mapping_nodes.to_csv('lineage_mapping_nodes.csv', index=False)

## Creating Table in Celonis

In [37]:
monitoring = data_pools.find("Sarvesh Monitoring Pool")

# 1. LINEAGE TABLE (STUDIO)
column_config_data_models = [
    ColumnTransport(column_name=c, column_type=ColumnType.STRING, field_length=100) 
    for c in df_studio.columns
]
monitoring.create_table(
    df=df_studio, 
    table_name='lineage_frontend', 
    column_config=column_config_data_models, 
    drop_if_exists=True
)
print(" Uploaded frontend lineage table")

# 2. MAPPING NODES
column_config_studio = [
    ColumnTransport(column_name=c, column_type=ColumnType.STRING, field_length=5000) 
    for c in df_studio.columns
]
monitoring.create_table(
    df=df_mapping_nodes, 
    table_name='mapping_nodes_frontend', 
    column_config=column_config_studio, 
    drop_if_exists=True
)
print(" Uploaded frontend mapping nodes table")

# 3. BRIDGE TABLE
column_config_studio = [
    ColumnTransport(column_name=c, column_type=ColumnType.STRING, field_length=5000) 
    for c in df_bridge.columns
]
monitoring.create_table(
    df=df_bridge, 
    table_name='bridge_lineage_mapping', 
    column_config=column_config_studio, 
    drop_if_exists=True
)
print(" Uploaded bridge table")

# 4. USED REPORT TABLE
column_config_studio = [
    ColumnTransport(column_name=c, column_type=ColumnType.STRING, field_length=5000) 
    for c in df_final.columns
]
monitoring.create_table(
    df=df_final, 
    table_name='usedtables_report_frontend', 
    column_config=column_config_studio, 
    drop_if_exists=True
)
print(" Uploaded used tables report")

 Uploaded frontend lineage table
 Uploaded frontend mapping nodes table
 Uploaded bridge table
 Uploaded used tables report
